In [1]:
import duckdb
import pandas as pd
import torch
from transformers import *
import numpy as np
#from keras.preprocessing.sequence import pad_sequences
from sklearn.metrics.pairwise import cosine_similarity

/usr/local/lib/python3.10/dist-packages/transformers/deepspeed.py:23: FutureWarning: transformers.deepspeed module is deprecated and will be removed in a future version. Please import deepspeed modules directly from transformers.integrations
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [2]:
print(torch.cuda.is_available())
print(torch.cuda.current_device())
print(torch.cuda.device_count())
print(torch.cuda.get_device_name(0))

True
0
1
Tesla T4


In [4]:
#filtering the ontology for level_3  research method and data analysis method.
ontology_df = pd.read_csv('ontology.csv')

#filtering the ontology for level_3  research method and data analysis method.

synonyms_df = pd.read_csv('synonyms.csv')


In [5]:
synonyms_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 31770 entries, 0 to 31769
Data columns (total 1 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   synonym  31770 non-null  object
dtypes: object(1)
memory usage: 248.3+ KB


In [6]:
synonyms_df["synonym"].nunique()

31770

###  'research method' is in the ontology but not in ent_id of the synonyms table

In [8]:
print('Before ', synonyms_df.shape)
synonyms_df.drop_duplicates(inplace=True)
print('After: ', synonyms_df.shape)
#resetting index to avoid issues
synonyms_df.reset_index(drop=True, inplace=True)

Before  (31770, 1)
After:  (31770, 1)


In [9]:
ontology_df.head()

,ent_id,definition,label,parent,level,related,level_1,level_2,level_3,level_4,level_5,level_6,level_7,level_8,level_9,level_10
0,research method,A research method is a systematic plan for con...,NaN,methodological entity,3.0,NaN,entity,methodological entity,research method,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,conceptual method,A conceptual method refers to a systematic app...,CONCEPTUAL_METHOD,research method,4.0,NaN,entity,methodological entity,research method,conceptual method,NaN,NaN,NaN,NaN,NaN,NaN
2,literature study,A literature study is a systematic and compreh...,CONCEPTUAL_METHOD,conceptual method,5.0,NaN,entity,methodological entity,research method,conceptual method,literature study,NaN,NaN,NaN,NaN,NaN
3,structured literature research,Structured literature research is a methodical...,CONCEPTUAL_METHOD,literature study,6.0,NaN,entity,methodological entity,research method,conceptual method,literature study,structured literature research,NaN,NaN,NaN,NaN
4,literature sample,A literature sample refers to a specific piece...,CONCEPTUAL_METHOD,structured literature research,7.0,NaN,entity,methodological entity,research method,conceptual method,literature study,structured literature research,literature sample,NaN,NaN,NaN


## Clean dataframe

In [10]:
# Assuming 'data_clean' is a DataFrame with a column named 'text'
data_clean_special = synonyms_df.copy() # Create a copy of the original dataframe

data_clean_special['text'] = data_clean_special['synonym']
del(data_clean_special['synonym'] )

In [11]:
data_ontology = ontology_df.copy()
data_ontology = data_ontology[['parent']]

data_ontology['text'] = data_ontology['parent']
del(data_ontology['parent'])

In [12]:
print('Before ', data_ontology.shape)
data_ontology.drop_duplicates(inplace=True)
print('After: ', data_ontology.shape)
#resetting index to avoid issues
data_ontology.reset_index(drop=True, inplace=True)
ontology_df = ontology_df[['parent']]
ontology_df.drop_duplicates(inplace=True)
ontology_df.reset_index(drop=True, inplace=True)

Before  (899, 1)
After:  (220, 1)


<ipython-input-12-2cb41990ed91>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ontology_df.drop_duplicates(inplace=True)


In [13]:
data_clean = data_clean_special["text"].to_list()
data_ontology = ontology_df["parent"].to_list()

In [14]:

# Assuming 'data_clean' is a list of lists where each inner list contains multiple text elements
#Since it is not a list I can just access the content of the row in the datafrme
data_clean_special =  [['[CLS] ' + ' '+ synonyms_df['synonym'].iloc[index] + ' [SEP]'] for index, inner_list in synonyms_df.iterrows()]
data_ontology =  [['[CLS] ' + ' '+ ontology_df['parent'].iloc[index] + ' [SEP]'] for index, inner_list in ontology_df.iterrows()]

In [15]:
# Examine the first list entry
print(data_clean_special[0])
print(data_ontology[0])

['[CLS]  QDA Miner Lite system [SEP]']
['[CLS]  methodological entity [SEP]']


In [16]:
#Apply BERT-tokenization

#Loading the DistilBert tokenizer

#dbert_tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')
scibert_tokenizer = AutoTokenizer.from_pretrained("allenai/scibert_scivocab_uncased")
#loading the pre-trained distilbert model
#model = DistilBertModel.from_pretrained('distilbert-base-uncased')
model = AutoModel.from_pretrained("allenai/scibert_scivocab_uncased")

Could not locate the tokenizer configuration file, will try to use the model config instead.
/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


config.json:   0%|          | 0.00/385 [00:00<?, ?B/s]

loading configuration file config.json from cache at /root/.cache/huggingface/hub/models--allenai--scibert_scivocab_uncased/snapshots/24f92d32b1bfb0bcaf9ab193ff3ad01e87732fc1/config.json
Model config BertConfig {
  "_name_or_path": "allenai/scibert_scivocab_uncased",
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "transformers_version": "4.41.2",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 31090
}



vocab.txt:   0%|          | 0.00/228k [00:00<?, ?B/s]

loading file vocab.txt from cache at /root/.cache/huggingface/hub/models--allenai--scibert_scivocab_uncased/snapshots/24f92d32b1bfb0bcaf9ab193ff3ad01e87732fc1/vocab.txt
loading file tokenizer.json from cache at None
loading file added_tokens.json from cache at None
loading file special_tokens_map.json from cache at None
loading file tokenizer_config.json from cache at None
loading configuration file config.json from cache at /root/.cache/huggingface/hub/models--allenai--scibert_scivocab_uncased/snapshots/24f92d32b1bfb0bcaf9ab193ff3ad01e87732fc1/config.json
Model config BertConfig {
  "_name_or_path": "allenai/scibert_scivocab_uncased",
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token

pytorch_model.bin:   0%|          | 0.00/442M [00:00<?, ?B/s]

loading weights file pytorch_model.bin from cache at /root/.cache/huggingface/hub/models--allenai--scibert_scivocab_uncased/snapshots/24f92d32b1bfb0bcaf9ab193ff3ad01e87732fc1/pytorch_model.bin
Some weights of the model checkpoint at allenai/scibert_scivocab_uncased were not used when initializing BertModel: ['cls.predictions.bias', 'cls.predictions.decoder.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializin

In [17]:
# Tokenize each text in data_clean_special
tokenized_synonym = [scibert_tokenizer.tokenize(text[0]) for text in data_clean_special]
tokenized_ontology = [scibert_tokenizer.tokenize(text[0]) for text in data_ontology]

# Examine the first list entry
print(tokenized_synonym[0])
print(tokenized_ontology[0])

['[CLS]', 'qd', '##a', 'miner', 'lit', '##e', 'system', '[SEP]']
['[CLS]', 'methodological', 'entity', '[SEP]']


In [18]:
# Calculate the lengths of tokenized texts
text_lengths_synonym = [len(tokens) for tokens in tokenized_synonym]
text_lengths_ontology = [len(tokens) for tokens in tokenized_ontology]
#determine maximum overall length
text_lengths = text_lengths_ontology + text_lengths_synonym


# Determine the 85th percentile for the maximum sequence length
max_seq_length_synonym = int(np.percentile(text_lengths_synonym, 85))
print("Maximum sequence length synonyms:", max_seq_length_synonym)

# Determine the 85th percentile for the maximum sequence length
max_seq_length_ontology = int(np.percentile(text_lengths_ontology, 85))
print("Maximum sequence length Ontology:", max_seq_length_ontology)

# Determine the 85th percentile for the maximum sequence length
max_seq_length = int(np.percentile(text_lengths, 85))
print("Maximum sequence length overall:", max_seq_length)

Maximum sequence length synonyms: 8
Maximum sequence length Ontology: 6
Maximum sequence length overall: 8


In [19]:
k_length_o = max_seq_length  # Replace max_seq_length with the determined maximum sequence length


In [20]:
def get_embeddings(terms):
    """
    Generates embeddings for a list of terms using a DistilBERT model.

    Args:
        terms: A list of strings or lists of strings (representing phrases).
        model: The DistilBERT model to use for generating embeddings.
        dbert_tokenizer: The DistilBERT tokenizer associated with the model.
        k_length_o: The maximum length for tokenization (used for padding/truncation).

    Returns:
        A dictionary mapping each term to its corresponding embedding vector.
    """
    embeddings = {}
    for term in terms:
        if isinstance(term, list):
            term = " ".join(term)

        # Tokenize & Convert to PyTorch tensors
        inputs = scibert_tokenizer(  # <--- Moved inside the loop
            term,
            add_special_tokens=True,
            padding='max_length',
            truncation=True,
            max_length=k_length_o,
            return_tensors='pt'
        )

        # Get the embedding (ensure model is on the correct device)
        with torch.no_grad():
            inputs = {key: val.to(model.device) for key, val in inputs.items()}  # Move to GPU if available
            outputs = model(**inputs)

        # Extract [CLS] token embedding
        embedding = outputs.last_hidden_state[:, 0, :].cpu().numpy()  # Move to CPU
        embeddings[term] = embedding[0]  # Remove batch dimension

    return embeddings

In [21]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Move the model to the GPU
model.to(device)

BertModel(
  (embeddings): BertEmbeddings(
    (word_embeddings): Embedding(31090, 768, padding_idx=0)
    (position_embeddings): Embedding(512, 768)
    (token_type_embeddings): Embedding(2, 768)
    (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): BertEncoder(
    (layer): ModuleList(
      (0-11): 12 x BertLayer(
        (attention): BertAttention(
          (self): BertSdpaSelfAttention(
            (query): Linear(in_features=768, out_features=768, bias=True)
            (key): Linear(in_features=768, out_features=768, bias=True)
            (value): Linear(in_features=768, out_features=768, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (output): BertSelfOutput(
            (dense): Linear(in_features=768, out_features=768, bias=True)
            (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
            (dropout): Dropout(p=0.1, inplace=False

In [22]:
embeddings_synonyms = get_embeddings(synonyms_df['synonym'])
embeddings_ontology = get_embeddings(ontology_df['parent'])

# Comparing New term

In [23]:
def is_synonym(new_term):
    """
    Compare a new_term with the list of synonyms and ontology terms stored using cosine similarity.

    Args:
        new term: a single term to be compared

    Returns:
        A string that tells if the term is a synonym to x word and the score or if there was not a synonym in the list
    """

    new_term_embds = get_embeddings(new_term)
    similarity_scores = {}
    overall_embeddings = embeddings_synonyms | embeddings_ontology
    for term, embedding in overall_embeddings.items():
        similarity = cosine_similarity(new_term_embds[new_term[0]].reshape(1, -1), embedding.reshape(1, -1))[0][0]
        similarity_scores[term] = similarity
    sorted_similarity_scores = dict(sorted(similarity_scores.items(), key=lambda item: item[1], reverse=True))
    if sorted_similarity_scores and list(sorted_similarity_scores.values())[0] > 0.7:
        top_synonym = list(sorted_similarity_scores.keys())[0]
        return f"is synonym to {top_synonym} with a similarity score of {list(sorted_similarity_scores.values())[0]}"
    else:
        return "No close synonyms found in the list."

In [24]:
similarity_scores = {}
embeddings = {}

# Tokenize & Convert to PyTorch tensors
inputs = scibert_tokenizer(  # <--- Moved inside the loop
            'apple',
            add_special_tokens=True,
            padding='max_length',
            truncation=True,
            max_length=k_length_o,
            return_tensors='pt'
        )

        # Get the embedding (ensure model is on the correct device)
with torch.no_grad():
            inputs = {key: val.to(model.device) for key, val in inputs.items()}  # Move to GPU if available
            outputs = model(**inputs)
# Extract [CLS] token embedding
embedding = outputs.last_hidden_state[:, 0, :].cpu().numpy()  # Move to CPU
embeddings['apple'] = embedding[0]  # Remove batch dimension

In [25]:
similarity_scores = {}

# Tokenize & Convert to PyTorch tensors
inputs = scibert_tokenizer(  # <--- Moved inside the loop
            'PLS',
            add_special_tokens=True,
            padding='max_length',
            truncation=True,
            max_length=k_length_o,
            return_tensors='pt'
        )

        # Get the embedding (ensure model is on the correct device)
with torch.no_grad():
            inputs = {key: val.to(model.device) for key, val in inputs.items()}  # Move to GPU if available
            outputs = model(**inputs)
# Extract [CLS] token embedding
embedding = outputs.last_hidden_state[:, 0, :].cpu().numpy()  # Move to CPU
embeddings['PLS'] = embedding[0]  # Remove batch dimension

In [26]:
similarity = cosine_similarity(embeddings['PLS'], embeddings['apple'])
print(similarity)

ValueError: Expected 2D array, got 1D array instead:
array=[-7.01133236e-02  1.52614927e+00  2.53698472e-02 -3.78919870e-01
 -4.42873836e-02 -6.43811598e-02  3.46115798e-01 -5.34968257e-01
  4.21226680e-01 -6.84282959e-01  8.35985363e-01  2.56860942e-01
  6.02248218e-03 -8.32898915e-01 -4.27876890e-01  3.53113621e-01
 -1.05500805e+00  2.16096975e-02  1.44859397e+00 -1.46257222e-01
 -1.44513607e-01  1.17746711e+00  1.54212434e-02 -1.27274990e-01
 -3.63162130e-01  4.02150273e-01 -8.62334788e-01  3.27129662e-01
 -1.43920958e-01  1.08492672e+00 -9.39331353e-02 -7.24775195e-01
  1.11520186e-01 -4.69090730e-01  1.35809332e-01 -8.46475303e-01
 -2.52943128e-01  4.35957052e-02  1.34669095e-01 -7.46790588e-01
  7.58891702e-01 -1.08231997e+00  1.29284370e+00 -7.26692975e-02
 -9.75235328e-02 -4.32223082e-01 -4.82730895e-01  1.79711446e-01
  3.41749936e-01 -3.15545470e-01 -1.75879255e-01  6.54595867e-02
 -3.98258746e-01  1.78731239e+00  1.27062511e+00 -3.09721883e-02
 -6.42239675e-02 -1.21336877e+0

In [33]:
new_term = ["apple"]
is_synonym(new_term)

'is synonym to PLS with a similarity score of 0.8802676200866699'

In [ ]:
is_synonym("apple")

In [28]:
new_term = ["Information Retrieval"]
is_synonym(new_term)

'is synonym to natural language processing with a similarity score of 0.8684603571891785'

In [32]:
import pickle

def save_embeddings_to_pickle(embeddings, filename):
    """
    Saves embeddings to a pickle file.

    Args:
        embeddings: A dictionary mapping terms to their corresponding embedding vectors.
        filename: The name of the pickle file to save the embeddings.
    """
    with open(filename, 'wb') as f:
        pickle.dump(embeddings, f)



In [30]:
save_embeddings_to_pickle(embeddings_ontology, 'embeddings_ontology.pkl')

In [31]:
save_embeddings_to_pickle(embeddings_synonyms, 'embeddings_synonyms.pkl')